In [ ]:
#==================================================================
#Program: DEVEL_0
#Version: 1.0
#Author: David Helminiak
#Date Created: August 18, 2024
#Date Last Modified: August 30, 2024
#Description: Re-implementation of a unified block classifier code for breast cancer data
#Operation: Move back into main program directory before running.
#Status: Deprecated - Needed to add abstraction; moved to DEVEL_1
#==================================================================

#Have the notebook fill more of the display width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:80% !important; }</style>"))

#Items otherwise covered when not running this code in a notebook
import tempfile
dir_tmp = tempfile.TemporaryDirectory(prefix='TMP_').name
configFileName = './CONFIG_0-TEST'

In [ ]:
#==================================================================
#NOTES
#==================================================================


#WSI images are expected to be .jpg files and block images are expected to be .tif files





In [ ]:
#==================================================================
#CONFIG PARAMETERS
#==================================================================


#Should classification of blocks and their originating WSI be performed
classifierTest = True

#Should classifier model components be saved/exported
classifierExport = True

#Should WSI be segmented into blocks and classified; generates data for RANDS
classifierWSI = True

#Should WSI preparation and block extraction overwrite previously generated files (default: False)
overwriteBlocksWSI = True






#CLASSIFICATION - BLOCK

#How many samples should be submitted in a batch through pytorch models used in classifier; only used for inferencing (default: 1)
#Incrementing in powers of 2 recommended to best leverage common GPU hardware designs
#For ResNet and DenseNet a 2080TI 11GB reliably handles 64x3x224x224 (resizeImageSize=[224,224]) or 16x3x400x400 (resizeImageSize=[])
batchsizeClassifier = 64

#Specify what dimensions images should be resized to; if no resizing is desired leave as [] (default: [224, 224])
#Leaving as [] will increase training time (also must change batchsizeClassifier), but can lead to improved scores
#Original implementation uses [224, 224], though with adaptive average pooling this isn't actually neccessary
resizeImageSize = [224, 224]

#If folds for XGB classifier cross validation should be manually defined (e.g. [['S1', 'S3'], ['S4', 'S2']]), else use specify number of folds to generate
#Default matches folds used in prior work (https://doi.org/10.3389/fonc.2023.1179025)
#Omits 6 available samples, with folds holding: 11, 12, 12, 12, 13 samples respectively; this may have been to better balance class distribution
#Presently, all available (non-excluded) samples (not just those in manualFolds) are currently used for training the exported/utilized final classifier
manualFolds = [['2', '9', '11', '16', '34', '36', '40', '54', '57', '60', '62'],
               ['17', '20', '23', '24', '28', '30', '33', '51', '52', '59', '63', '66'], 
               ['12', '14', '22', '26', '35', '44', '45', '47', '49', '53', '56', '68'], 
               ['4', '5', '8', '10', '25', '27', '29', '37', '42', '48', '50', '69'], 
               ['7', '15', '19', '31', '43', '46', '55', '58', '61', '64', '65', '67', '70']]

#Which pre-trained weight sets should be used for ResNet and DenseNet model: 'IMAGENET1K_V1', 'IMAGENET1K_V2'
#Unclear which weights were used for TensorFlow ResNet50 variant in original implementation, but V2 improves the score a bit when using resizeImageSize = []
weightsResNet = 'IMAGENET1K_V2'

#Which pre-trained weight sets should be used for DenseNet model: 'IMAGENET1K_V1'
weightsDenseNet = 'IMAGENET1K_V1'

#What ratio of malignant to benign blocks should be used to label a whole WSI as malignant
#Unknown what the original work used for this value, but chose a value (in range of 0.12-0.19) that can replicate prior results
thresholdWSI = 0.15


#CLASSIFICATION - WSI

#When splitting WSI images, what size should the resulting blocks be (default: [400, 400])
blockSize = [400, 400]






#RARELY CHANGED

#Define labels used for normal/benign tissue
#'a': normal adipose.
#'s': normal stroma tissue excluding adipose.
#'o': other normal tissue including parenchyma, adenosis, lobules, blood vessels, etc.
benignLabels = ['a', 's', 'o', 
                'normal', 
                'fibroadenoma', 
                'normal breast tissue'
               ]

#Define labels used for malignant tissue
#'d': IDC tumor
#'l': ILC tumor
#'ot': other tumor areas including DCIS, biopsy site, and slightly defocused tumor regions.
malignantLabels = ['d', 'l', 'ot', 
                   'breast cancer', 
                   'IMC: Invasive Mucinous', 
                   'IDC', 
                   'IMC: IDC with lobular features', 
                   'IMC: ILC', 
                   'ILC', 
                   'Invasive lobular carcinoma'
                  ]

#Define labels used for tissues to be excluded
#'ft': defocused but still visually tumor-like areas.
#'f': severly out-of-focusing areas. 
#'b': background. 
#'e': bubbles.
#Other labels used that were excluded from original work, so doing so here as well
excludeLabels = ['ft', 'f', 'b', 'e',
                'lymph node',
                 'tongue tumor: invasive squamous cell carcinoma', 
                 'High grade carcinoma in lymph node', 
                 'osteosarcoma from mandible',
                 'normal tongue'
                ]





#Already in setup CONFIG (avoid duplication during copy)
#==================================================================
#Should parallelization calls be used to leverage multithreading where able
parallelization = True

#If parallelization is enabled, how many CPU threads should be used in Ray tasks? (0 will use any/all available)
#Recommend starting at half of the available system threads if using hyperthreading,
#or 1-2 less than the number of system CPU cores if not using hyperthreading.
#Adjust to where the CPU just below 100% usage during parallel operations 
availableThreads = 16

#Which GPU(s) devices should be used (last specified used for data processing and model training); (default: [-1], any/all available; CPU only: [])
gpus = [-1]

#Should training/validation data be entirely stored on GPU (default: True; improves training/validation efficiency, set to False if OOM occurs)
storeOnDevice = True
   
#RNG seed value to ensure run-to-run consistency (-1 to disable)
manualSeedValue = 0

debugMode = False

asciiFlag = False



In [ ]:
#==================================================================
#MAIN - Run codes that have already been completed
#==================================================================

exec(open("./CODE/EXTERNAL.py", encoding='utf-8').read())

exec(open("./CODE/COMPUTE.py", encoding='utf-8').read())

#Turn off image size checking; note that this can allow for decompression bomb DOS attacks if an untrusted image ends up as an input
Image.MAX_IMAGE_PIXELS = None




In [ ]:
#==================================================================
#MODEL_CLASS
#==================================================================

#Load and preprocess data image files
class DataPreprocessing_Classifier(Dataset):
    def __init__(self, filenames):
        super().__init__()
        self.filenames = filenames
        self.numFiles = len(self.filenames)
        if len(resizeImageSize) > 0:
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                v2.Resize(tuple(resizeImageSize)), 
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        else: 
             self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        
    def __getitem__(self, index): return self.transform(Image.open(self.filenames[index]))

    def __len__(self): return self.numFiles

#Load/synchronize data labeling, drop excluded rows, and extract relevant metadata
def loadMetadata(filename):
    metadata = pd.read_excel(filename, header=None, names=['name', 'label'], converters={'name':str,'label':str})
    metadata['label'].replace(benignLabels, benignLabel, inplace=True)
    metadata['label'].replace(malignantLabels, malignantLabel, inplace=True)
    metadata['label'].replace(excludeLabels, excludeLabel, inplace=True)
    metadata = metadata.loc[metadata['label'] != excludeLabel]
    return np.array(metadata['name']), np.array(metadata['label'])

#Compute metrics for a classification result and visualize/save them as needed
def computeClassificationMetrics(labels, predictions, displayLabels, filename):
    classificationReport = classification_report(labels, predictions, target_names=displayLabels)
    confusionMatrix = confusion_matrix(labels, predictions)
    tn, fp, fn, tp = confusionMatrix.ravel()
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    print(classificationReport)
    print('Accuracy: ', accuracy, '\tSensitivity: ', sensitivity, '\tSpecificity: ', specificity)
    displayCM = ConfusionMatrixDisplay(confusionMatrix, display_labels=displayLabels)
    displayCM.plot(cmap='Blues'); plt.show(); plt.close()
    

In [ ]:
#==================================================================
#INTERNAL_DIRECTORY
#==================================================================

#Indicate and setup the destination folder for results of this configuration
destResultsFolder = './RESULTS_'+os.path.splitext(os.path.basename(configFileName).split('_')[1])[0]

#If the folder already exists, either remove it, or append a novel value to it
if os.path.exists(destResultsFolder):
    if not preventResultsOverwrite: 
        shutil.rmtree(destResultsFolder)
    else: 
        destinationNameValue = 0
        destResultsFolder_Base = copy.deepcopy(destResultsFolder)
        while True:
            destResultsFolder = destResultsFolder_Base + '_' + str(destinationNameValue)
            if not os.path.exists(destResultsFolder): break
            destinationNameValue += 1


#Set a base model name for the specified configuration
modelName = 'model_RANDS_'

#Data input directories/files
dir_data = '.' + os.path.sep + 'DATA' + os.path.sep

#Training and testing of block classification
dir_dataBlocks = dir_data + 'BLOCKS' + os.path.sep
dir_inputsBlocks = dir_dataBlocks + 'INPUTS' + os.path.sep
file_labelsBlocks = dir_inputsBlocks + 'Patch_list.xlsx'

#Testing of WSI classification and generation of RANDS data
dir_dataWSI = dir_data + 'WSI' + os.path.sep
dir_inputsWSI = dir_dataWSI + 'INPUTS' + os.path.sep
dir_preparedWSI = dir_dataWSI + 'RESULTS_PREPARED_WSI' + os.path.sep
dir_blocksWSI = dir_dataWSI + 'RESULTS_BLOCKS' + os.path.sep
file_labelsWSI = dir_inputsWSI + 'WSI_list.xlsx'


#Remove files/folders that are to be overwritten
if overwriteBlocksWSI and os.path.exists(dir_blocksWSI): 
    shutil.rmtree(dir_blocksWSI)
    os.makedirs(dir_blocksWSI)
    shutil.rmtree(dir_preparedWSI)
    os.makedirs(dir_preparedWSI)


In [ ]:
#==================================================================
#DATA_PROCESSING
#==================================================================

#Define generalize labels used in place of the original, specific tissue labels
benignLabel, malignantLabel, excludeLabel = 0, 1, 2

#If features used in training the XGB Classifier model are needed (either for export or testing)
if classifierExport or classifierTest: 

    #Load and process metadata for available blocks
    blockNamesAll, blockLabelsAll = loadMetadata(file_labelsBlocks)
    blockSampleNamesAll = np.array([re.split('PS|_', blockName)[1] for blockName in blockNamesAll])
    blockFilenamesAll = [dir_inputsBlocks + 'S' + blockSampleNamesAll[blockIndex] + os.path.sep + blockNamesAll[blockIndex] + '.tif' for blockIndex in range(0, len(blockNamesAll))]

    #Prepare data for use with PyTorch ResNet50 and DenseNet169 models
    device = f"cuda:{gpus[-1]}" if len(gpus) > 0 else "cpu"
    torchDevice = torch.device(device)
    blockData = DataPreprocessing_Classifier(blockFilenamesAll)
    blockDataloader = DataLoader(blockData, batch_size=batchsizeClassifier, num_workers=0, shuffle=False)#, pin_memory=True)
    numBlockData = len(blockDataloader)

    #==================================================================
    #FEATURE EXTRACTION
    #==================================================================

    #Load pretrained resnet50 model and set to evaluation mode
    model_ResNet = models.resnet50(weights=weightsResNet).to(device)
    _ = model_ResNet.train(False)

    #If classifier is to be exported, do so here
    #if classifierExport: 
    
    #Extract features for each batch of sample block images
    blockFeaturesAll = []
    for data in tqdm(blockDataloader, total=numBlockData, desc='Feature Extraction', leave=True, ascii=asciiFlag):
        blockFeaturesAll += model_ResNet(data.to(torchDevice)).detach().cpu().tolist()

    #Convert list of features to an array
    blockFeaturesAll = np.asarray(blockFeaturesAll)

    #Clear the ResNet model from the GPU
    del model_ResNet
    if len(gpus) > 0: torch.cuda.empty_cache() 


#==================================================================
#CLASSIFICATION - XGBClassifier
#==================================================================

#If the classifier model should be tested
if classifierTest: 

    #Allocate samples to folds for cross validation
    if type(manualFolds) != list: folds = [array.tolist() for array in np.array_split(np.random.permutation(np.unique(blockSampleNamesAll)), manualFolds)]
    else: folds = manualFolds
    numFolds = len(folds)

    #Split block features into specified folds, keeping track of originating indices and matched labels
    foldsFeatures, foldsLabels, foldsSampleNames, foldsIndices = [], [], [], []
    for fold in folds:
        blockIndices = np.concatenate([np.where(blockSampleNamesAll == sampleName)[0] for sampleName in fold])
        foldsIndices.append(blockIndices)
        foldsFeatures.append(list(blockFeaturesAll[blockIndices]))
        foldsLabels.append(list(blockLabelsAll[blockIndices]))
        foldsSampleNames.append(list(blockSampleNamesAll[blockIndices]))

    #Collapse indices, labels, and sample names for later evaluation of the fold data
    blockIndices = np.concatenate(foldsIndices)
    blockLabels = np.concatenate(foldsLabels)
    blockSampleNames = np.concatenate(foldsSampleNames)

    #Check class distribution between the folds
    #print('B\t M \t Total')
    #for foldNum in range(0, numFolds):
    #    print(np.sum(np.array(foldsLabels[foldNum]) == 0),'\t', np.sum(np.array(foldsLabels[foldNum]) == 1), '\t', len(foldsLabels[foldNum]))

    #Test on each fold, while training with all remaining, saving predictions for each block
    #Note that GPU and CPU implementations of XGBClassifier use different algorithms and will yield different results
    blockPredictions = []
    for foldNum in tqdm(range(0, numFolds), desc='Block Classification', leave=True, ascii=asciiFlag):
        model_XGBClassifier = XGBClassifier(device=device)
        _  = model_XGBClassifier.fit(np.concatenate(foldsFeatures[:foldNum]+foldsFeatures[foldNum+1:]), np.concatenate(foldsLabels[:foldNum]+foldsLabels[foldNum+1:]))
        blockPredictions += model_XGBClassifier.predict(np.asarray(foldsFeatures[foldNum])).tolist()

    #Convert list of predictions to an array
    blockPredictions = np.asarray(blockPredictions)

    #Evaluate results
    print('Initial Block Classifier Results')
    computeClassificationMetrics(blockLabels, blockPredictions, ['Benign', 'Malignant'], None)

    #==================================================================
    #CLASSIFICATION - GradCam++
    #==================================================================
    #This section focues on reducing false positives on WSI classification and/or for novelty...
    #It doesn't improve the per-block classification results and wouldn't be recommended for use in RANDS implementation...

    #Load pre-trained DenseNet
    model_DenseNet = models.densenet169(weights=weightsDenseNet)

    #Replace the in-built classifier; unclear how this structure and these hyperparameters were determined
    model_DenseNet.classifier = nn.Sequential(
        nn.Linear(model_DenseNet.classifier.in_features, 256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 2),
        nn.LogSoftmax(dim=1)
    )
    model_DenseNet = model_DenseNet.to(torchDevice)
    _ = model_DenseNet.train(False)

    #Create GradCAMPlusPlus model; see https://github.com/jacobgil/pytorch-grad-cam for additional models and options
    model_GradCamPlusPlus = GradCAMPlusPlus(model=model_DenseNet, target_layers=[model_DenseNet.features[-1]])

    #If visualization of block saliency maps is going to be performed, then will need to define un-normalized transform for overlay target
    if len(resizeImageSize)>0: transform = transforms.Compose([transforms.ToTensor(), v2.Resize(tuple(resizeImageSize))])
    else: transform = transforms.Compose([transforms.ToTensor()])

    #Compute confidence weights for each block
    blockWeights = []
    for data in tqdm(blockDataloader, total=numBlockData, desc='Weight Computation', leave=True, ascii=asciiFlag):

        #Generate saliency maps for each block
        blockSaliencyMaps = model_GradCamPlusPlus(input_tensor=data, targets=None)

        #For current GradCamPlusPlus implementation, must manually clear internal copy of the outputs from GPU cache to prevent OOM
        del model_GradCamPlusPlus.outputs 
        if len(gpus) > 0: torch.cuda.empty_cache()

        #Visualization of block saliency maps
        #for index, blockFilename in enumerate(blockFilenamesAll[batchNum*batchsizeClassifier:(batchNum+1)*batchsizeClassifier]):
        #    blockImage = np.moveaxis(transform(Image.open(blockFilename)).numpy(), 0, -1)
        #    blockSaliencyMap = blockSaliencyMaps[index]
        #    blockOverlaid = show_cam_on_image(blockImage, blockSaliencyMap, use_rgb=True)
        #    plt.imshow(blockOverlaid); plt.show(); plt.close()

        #Compute regional importance for each block as the average saliency map values
        blockImportances = np.mean(blockSaliencyMaps, axis=(1,2))

        #Threshold regional importance values by 0.25 to get block weights
        blockWeights += list(np.where(blockImportances<0.25, 0, blockImportances))

    #Convert list of new predictions to an array
    blockWeights = np.asarray(blockWeights)

    Tracer()
    
    #Multiply the block predictions (using -1 for benign and +1 for malignant) by the matching block weights
    blockPredictionsNew = np.where(blockPredictions==0, -1, 1)*blockWeights[blockIndices]

    #Identify any positive values as malignant and the remaining as benign
    blockPredictionsNew = np.where(blockPredictionsNew>0, 1, 0)

    #Evaluate new per-block results
    print('Updated Block Classifier Results')
    computeClassificationMetrics(blockLabels, blockPredictionsNew, ['Benign', 'Malignant'], None)


    #==================================================================
    #CLASSIFICATION - WSI
    #==================================================================

    sampleLabels, samplePredictions, samplePredictionsNew = [], [], []
    for sampleName in np.unique(blockSampleNames):

        #Find indices of samplename in sampleNames
        sampleIndices = np.where(blockSampleNames == sampleName)[0]

        #Determine if the number of malignant block predictions exceeds the configured threshold
        sampleLabels.append((np.mean(blockLabels[sampleIndices]) >= thresholdWSI)*1)
        samplePredictions.append((np.mean(blockPredictions[sampleIndices]) >= thresholdWSI)*1)
        samplePredictionsNew.append((np.mean(blockPredictionsNew[sampleIndices]) >= thresholdWSI)*1)

    #Convert list of WSI labels/predictions to arrays
    sampleLabels = np.asarray(sampleLabels)
    samplePredictions = np.asarray(samplePredictions)
    samplePredictionsNew = np.asarray(samplePredictionsNew)

    #Evaluate original WSI results
    print('Initial WSI Classifier Results')
    computeClassificationMetrics(sampleLabels, samplePredictions, ['Benign', 'Malignant'], None)

    #Evaluate updated WSI results
    print('Updated WSI Classifier Results')
    computeClassificationMetrics(sampleLabels, samplePredictionsNew, ['Benign', 'Malignant'], None)
    
#If classifier model components (other than ResNet) are to be exported
if classifierExport:
    
    #Train the XGBClassifier on all available block data
    model_XGBClassifier = XGBClassifier(device=device)
    _  = model_XGBClassifier.fit(blockFeaturesAll, blockLabelsAll)
    
    #Export/Store the XGBClassifier model here

    #Clear the XGBClassifier model from the GPU
    del model_XGBClassifier
    if len(gpus) > 0: torch.cuda.empty_cache() 

        


In [ ]:

#If WSI should be segmented into blocks and classified
if classifierWSI:

    #Either load blockFilenamesAll or split WSI to blocks and obtain such
    #Original work resized WSI before splitting into blocks, which potentially introduced inconsistent artifacting from altering the native aspect ratio
    if not overwriteBlocksWSI:
        blockFilenamesAll = [natsort.natsorted(glob.glob(dir_sampleBlocksWSI+os.path.sep+'*.tif')) for dir_sampleBlocksWSI in natsort.natsorted(glob.glob(dir_blocksWSI+'*'))]
    else: 
        
        #Load and process metadata for available samples; could probably parallelize this if given time to do so
        sampleNamesAll, sampleLabelsAll = loadMetadata(file_labelsWSI)
        sampleFilenamesAll = [dir_inputsWSI + sampleNamesAll[sampleIndex] + '.jpg' for sampleIndex in range(0, len(sampleNamesAll))]

        blockFilenamesAll = []
        for filename in tqdm(sampleFilenamesAll, desc='WSI Block Extraction', leave=True, ascii=asciiFlag):

            #Extract base sample name
            sampleName = os.path.basename(filename).split('.jpg')[0]

            #Load WSI image
            imageWSI = np.asarray(Image.open(filename))

            #Equalize image brightness with historgram equalization in YUV space and extract foreground with Otsu; no improvement, leaving for reference
            #y, cr, cb = cv2.split(cv2.cvtColor(imageWSI, cv2.COLOR_RGB2YCrCb))
            #imageWSI = cv2.cvtColor(cv2.merge((cv2.equalizeHist(y), cr, cb)), cv2.COLOR_YCR_CB2RGB)
            #mask = cv2.threshold(cv2.cvtColor(imageWSI, cv2.COLOR_RGB2GRAY),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
            
            #Crop image to the largest foreground area
            mask = cv2.threshold(cv2.cvtColor(imageWSI, cv2.COLOR_RGB2GRAY),0,255,cv2.THRESH_BINARY)[1]
            contour = max(cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0], key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(contour) 
            imageWSI = copy.deepcopy(imageWSI)[y:y+h, x:x+w]

            #Pad the image as needed (as symmetrially as possible) for an even division by the specified block size; compute numBlocks per row/column
            padHeight = (int(np.ceil(imageWSI.shape[0]/blockSize[0]))*blockSize[0])-imageWSI.shape[0]
            padWidth = (int(np.ceil(imageWSI.shape[1]/blockSize[1]))*blockSize[1])-imageWSI.shape[1]
            padTop, padLeft = padHeight//2, padWidth//2
            padBottom, padRight = padTop+(padHeight%2), padLeft+(padWidth%2)
            imageWSI = np.pad(imageWSI, ((padTop, padBottom), (padLeft, padRight), (0, 0)))
            numBlocksRow, numBlocksCol = imageWSI.shape[0]//blockSize[0], imageWSI.shape[1]//blockSize[1]

            #Save processed WSI for ease of later visualization; Pillow with .tif extension saves perfectly lossless  (no loss of quality)
            Image.fromarray(imageWSI).save(dir_preparedWSI + 'S' + sampleName + '.tif')

            #Split the WSI into blocks and flatten
            imageWSI = imageWSI.reshape(numBlocksRow, blockSize[0], numBlocksCol, blockSize[1], imageWSI.shape[2]).swapaxes(1,2)
            imageWSI = imageWSI.reshape(-1, imageWSI.shape[2], imageWSI.shape[3], imageWSI.shape[4])

            #Setup directory to store blocks and generate filenames/locations for each
            dir_sampleBlocksWSI = dir_blocksWSI + 'S' + sampleName + os.path.sep
            if not os.path.exists(dir_sampleBlocksWSI): os.makedirs(dir_sampleBlocksWSI)
            blockFilenames = [dir_sampleBlocksWSI+'PS'+sampleName+'_'+str(rowNum)+'_'+str(colNum)+'.tif' for rowNum in range(0, numBlocksRow) for colNum in range(0, numBlocksCol)]
            blockFilenamesAll.append(blockFilenames)

            #Store blocks to disk; Pillow with .tif extension saves perfectly lossless  (no loss of quality)
            for index in range(0, len(blockFilenames)): Image.fromarray(imageWSI[index]).save(blockFilenames[index])

        del imageWSI
        cleanup()



In [ ]:

#Setup data loader

#Load ResNet and extract features

#Load XGB Classifier and classify the blocks; if background then set as benign...

#Apply saliency mapping?

#Output versions with and without 'fusion' method...


In [ ]:
#Recombine split blocks (numBlocksRow, numBlocksCol, blockSize[0], blockSize[1], 3)
#mergeWSI = splitWSI.swapaxes(1,2)
#mergeWSI = mergeWSI.reshape(mergeWSI.shape[0]*mergeWSI.shape[1], mergeWSI.shape[2]*mergeWSI.shape[3], mergeWSI.shape[4])
